<a href="https://colab.research.google.com/github/dimidagd/gists/blob/main/CIFA_10_classification_k_nn_dinov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO

# Load DINOv2 model
models= []
# DINOv2 with registers
models += [torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg').to("cuda")]
#models += [torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14_reg')]
#models += [torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14_reg')]
#models += [torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg')]

[model.eval() for model in models]

# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load image from URL or file
def load_image(image_path):
    if image_path.startswith('http'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(image_path)
    return image

# Function to get image embeddings
def get_image_embeddings(model, image_path):
    image = load_image(image_path)
    image = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embeddings = model(image.to("cuda"))
    return embeddings

# Example usage
for model in models:
  image_path = 'https://picsum.photos/200/300'  # Replace with your image path or URL
  embeddings = get_image_embeddings(model, image_path)
  print(embeddings.shape)


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


torch.Size([1, 384])


In [ ]:
model = models[0]
from tqdm.notebook import tqdm
import numpy as np
def extract_features(dataloader):
    all_features = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            outputs = model(inputs.to("cuda"))
            all_features.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    return np.vstack(all_features), np.hstack(all_labels)

# Extract features
train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

print(f"Train feature shape: {train_features.shape}, Train labels shape: {train_labels.shape}")
print(f"Test feature shape: {test_features.shape}, Test labels shape: {test_labels.shape}")

  0%|          | 0/782 [00:00<?, ?it/s]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
# Train the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=5)

print("Training k-NN classifier")
knn.fit(train_features, train_labels)

# Evaluate the classifier
accuracy = knn.score(test_features, test_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Predict on the test set
print("Predicting on the test set")
y_pred = knn.predict(test_features)

# Print classification report
report = classification_report(test_labels, y_pred, target_names=class_names)
print(report)